# Pretraining 2: GPT-2 355M 

we are going to go big here 

https://huggingface.co/datasets/PatrickHaller/fineweb-3B

This dataset is over 8 GB. So hopefully your internet connection is fast enough.

In [ ]:
%%bash
if [ ! -f data/fineweb-3b/README.md ]; then
    echo "Data set not yet downloaded. Downloading now..."
    git clone https://huggingface.co/datasets/PatrickHaller/fineweb-3B data/fineweb-3b
else
    echo "Data set is downloaded."
fi 

mkdir -p data/fineweb-3b/text

THe data set is in parqet format. so we will need to write a conversion script that will convert parquet to CSV text

In [ ]:
!pip install pandas
!pip install pyarrow

In [ ]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)

base_path = "data/fineweb-3b/data"

all_files = os.listdir(base_path)

output_path = "data/fineweb-3b/text"

for i, filename in enumerate(all_files):
    print(filename)

In [ ]:
for i, filename in enumerate(all_files):
    fullpath = f"{base_path}/{filename}"

    df = pd.read_parquet(fullpath)

    data = df["text"].to_csv(index=False)
    
    print(data)

    break


Writing all files to output. The order at which these are processed isn't really important. But expect it to be large.

In [ ]:

for i, filename in enumerate(all_files):
    fullpath = f"{base_path}/{filename}"

    df = pd.read_parquet(fullpath)

    data = df["text"].to_csv(index=False)

    with open(f"{output_path}/data-{i}.txt", "w") as f:
        # Skip the first line
        if data.startswith("text\n"):
            data = data[5:]
        
        f.write(data)

    print(f"Processed: {filename}")
    

In [ ]:
with open(f"{output_path}/data-2.txt") as f:
  print(f.readline()) # this is the header
  print(f.readline())

Now that we have all this data, we will need to create batches for the training set and validation data set. But this can be quite large. Just for this data set we do have enough memory to hold everything with 64 GB in this system. So we will be lazy and just load everything into memory and will do a split.

I think Jupyter will run out of memory, so we will have to do this differently. I wrote a utility in C to quickly concatenate the raw text. 

In [ ]:
%%bash
../text-builder/text-builder data/fineweb-3b/text data/fineweb-3b/raw_data.txt

In [ ]:
%%bash
../text-splitter/text-splitter data/fineweb-3b/raw_data.txt data/fineweb-3b/train_data.txt data/fineweb-3b/val_data.txt